In [ ]:
# !pip install monai
# !pip install torchvision
# !pip install -U Setuptools
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
# !pip install adabelief-pytorch==0.2.0

In [ ]:
# !pip install ipywidgets widgetsnbextension
# !jupyter nbextension enable --py widgetsnbextension

# 訓練 ECA-NFNet

In [ ]:
!nvidia-smi

In [ ]:
import logging
import os
import sys
import tempfile
import glob
import time
import matplotlib.pyplot as plt
import numpy as np

import setuptools
import torch
import torchvision
from PIL import Image
from torch.utils.data import DataLoader #只有dataloader用torch的
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChanneld,
    AsDiscrete,
    Compose,  
    LoadImaged,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
    AsChannelFirstd,
    Resized,
    SaveImage,
    Resize,
)

# Monai 狀態

In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Set Data folder
> 設定大資料夾的位置，裡頭需含有`Train_Images`與`Train_Annotations_png`兩資料夾

```
SEG_Train_Datasets
        |-> Train_Images
        |-> Train_Annotations_png
```

In [ ]:
# Set the Data folder
data_path = './SEG_Train_Datasets/'
os.listdir(data_path)

# Load train files
> 將一張image對應一張mask，並且用dictionary包起來，最後再用一個list將所有組包起來，並且藉由list indice將Training Data與Validation Data分出來

In [ ]:
for_test = 237


tempdir = data_path + "Train_Images"
train_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))

tempdir = data_path + "Train_Annotations_png"
train_segs = sorted(glob.glob(os.path.join(tempdir, "*.png")))

# Training data
train_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[for_test:], train_segs[for_test:])]
print(f" {len(train_images[for_test:])} train_images and {len(train_segs[for_test:])} train_segs")

# validation data
val_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[:for_test], train_segs[:for_test])]
print(f" {len(train_images[:for_test])} val_images and {len(train_segs[:for_test])} val_segs")

In [ ]:
# check
train_files[:5]

# Define Trasform for image and segmentation
> 製作transforms，注意有些是作用在image與mask上，但有些只作用在image

### 特別附註
```
Monai的Resize、Resized，為作用於input的第三與第四維，代表你的圖片必須要是三維的，否則會出錯
正確input : 
    (1, 1, 1716, 942)  --> Resize 800x800 --> (1, 1, 800, 800)

錯誤input :
    (1, 1716, 942)  --> Resize 800x800 --> (1, 1716, 800, 800)  
```


In [ ]:
# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        Resized(keys=["img", "seg"], spatial_size=[800, 800]),
        RandRotate90d(keys=["img", "seg"], prob=0.3, spatial_axes=[0, 1]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)

# Create DataLoader for train and validation data
> 重要參數 `batch_size`，需依照自己的GPU記憶體大小設定

In [ ]:
# batch_size
batch_size = 24
# create a training data loader
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)

# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
train_loader = DataLoader(
    train_ds,
    batch_size= batch_size,
    shuffle=True,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)

# create a validation data loader
val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size= batch_size, collate_fn=list_data_collate)



# Define metric and post-processing
> 設定DiceMetric，與output用的transform，這部分則使用老師的example code中所寫的內容


In [ ]:
dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)

#output transform
post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])

# Set environment
> 設定GPU數量，以及啟用GPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3'

In [ ]:
# check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Create Visualize Function

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, cmap= 'gray')
    plt.show()

# Built Model

> Decoder : DeepLabV3Plus
>
> Encoder : tu-eca_nfnet_l2

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
# 用來設定模型存檔的名稱
encode_name = 'tu-eca_nfnet_l2_DeepLabV3Plus_origin'

In [ ]:
aux_params=dict(
    pooling='avg',             # one of 'avg', 'max'
    dropout=0.4,               # dropout ratio, default is None
    activation=None,           # activation function, default is None
    classes=1,                 # define number of output labels
)


encodes = 'tu-eca_nfnet_l2'
model_no = smp.DeepLabV3Plus(encodes, aux_params=aux_params)

#  因為此為多GPU運算，所以必須使用`torch.nn`的`DataParallel`

In [ ]:
model = torch.nn.DataParallel(model_no).to(device)

In [ ]:
# optimizer
from adabelief_pytorch import AdaBelief
optimizer = AdaBelief(model.parameters(), lr=1e-4, eps=1e-16, betas=(0.9, 0.99), weight_decouple = True, rectify = False, weight_decay = 1e-4)
loss_function = monai.losses.DiceLoss(sigmoid=True)

# 正式訓練
> 這部分是使用老師提供的example下去微調，微調部分如總epoch、存模型的路徑等等

In [ ]:
#### start a typical PyTorch training
total_epochs = 500
val_interval = 1
best_metric = 100   #存best metric
best_metric_epoch = -1  #存best epoch
metric_values = list()
writer = SummaryWriter()  
train_loss = []
test_loss = []
lr_set  = []

for epoch in range(total_epochs):

    print("-" * 10)
    print(f"epoch {epoch + 1}/{total_epochs}")

    # 開起訓練模式
    model.train()
    epoch_loss = 0
    step = 0

    #計時
    time_start = time.time()

    for toto, batch_data in enumerate(train_loader):

        step += 1

        # 訓練過程
        time_end = time.time()
        epoch_len = len(train_ds) // train_loader.batch_size   #多少epoch
        print(f"on {step}/{epoch_len} GOGOGO   Use {'%.3f'%(time_end - time_start)}s   ||{'='*step+'>'+'-'*(epoch_len-step)}||" , end='\r')


        inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
        optimizer.zero_grad()    
        outputs, label = model(inputs) #forward    
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += float(loss.item())
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)


    epoch_loss /= step
    train_loss.append(epoch_loss)
    local_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())    
    print("\n", f"{local_time} epoch {epoch + 1} training average loss: {epoch_loss:.4f}") 

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            steps = 0
            loss_val = 0

            for val_data in val_loader:
                val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                
                val_outputs, val_out_label = model(val_images) #forward
                val_outputs = Resize([-1, 1716, 942])(val_outputs)

                val_loss = monai.losses.DiceLoss(sigmoid=True)(val_outputs, val_labels)
                loss_val += float(val_loss)


                #output transform
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                

                if  steps == 3 or steps == 5:
                    print("val loss", val_loss)
                    visualize( 
                        image=val_images[0].cpu().permute(1,2,0), 
                        ground_truth_mask=val_labels[0].cpu().permute(1,2,0), 
                        predicted_mask=val_outputs[0].cpu().permute(1,2,0)

                    )  
                steps += 1
                dice_metric(y_pred=val_outputs, y=val_labels)


            # aggregate the final mean dice result
            print("val_loss = ", loss_val / steps)
            val_loss_ave = loss_val / steps

            # reset the status for next validation round
            dice_metric.reset()
            metric_values.append(val_loss_ave)
            test_loss.append(val_loss_ave)

            if val_loss_ave < best_metric:
                best_metric = val_loss_ave
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), f"{encode_name}.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current val mean dice loss: {:.4f} best val mean dice loss: {:.4f} at epoch {}".format(
                    epoch + 1, val_loss_ave, best_metric, best_metric_epoch
                )
            )
            
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

In [ ]:
# plot

fig, ax = plt.subplots(1, figsize= (16, 8))
ax.plot(np.arange(0, len(train_loss), 1), train_loss, label= 'train_loss', linewidth= 3)

ax.plot(np.arange(0, len(test_loss), 1), test_loss, label= 'test_loss', linewidth= 3)
ax.set_title(f'[{model.__class__.__name__} + {encode_name}_{ver}] Train_Test_loss')
ax.set_ylabel('Loss', fontsize= 18)
plt.legend()

ax1 = ax.twinx()
ax1.plot(np.arange(0, len(lr_set), 1), lr_set, label= 'Learning Rate', c= 'black', linestyle= '--')
ax1.set_ylabel('Learning Rate', fontsize= 18)
ax.set_xlabel('epochs', fontsize= 18)

bbox = dict(boxstyle="round", fc="0.9")

plt.legend(loc= 'upper left')


plt.text(0.5, 0.935, 'best epochs %d, val_loss= %.4f'%(best_metric_epoch, best_metric), transform=ax.transAxes, bbox= bbox, fontsize= 18)

plt.show()

fig.savefig(f'{model.__class__.__name__} + {encode_name}_{ver}_Train_Test_loss.png')

# Load Model並重現Validation Data

In [ ]:
model.load_state_dict(torch.load(f"{encode_name}.pth"))


In [ ]:
model.eval()
dice_metric.reset()
with torch.no_grad():
    val_images = None
    val_labels = None
    val_outputs = None
    for val_data in val_loader:
        val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)

        val_outputs, val_out_label = model(val_images) #forward
        val_outputs = Resize([-1, 1716, 942])(val_outputs)
        val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                

        # compute metric for current iteration
        dice_metric(y_pred=val_outputs, y=val_labels)
        print(dice_metric.aggregate())
        
    # aggregate the final mean dice result
    metric = dice_metric.aggregate().item()
    print("metric = ", metric)
    # reset the status for next validation round
    dice_metric.reset()

# Predict Public dataset

In [ ]:
tempdir = "./Public_Image/"
test_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))

print(f" {len(test_images)} test_images")

test_files = [{"img": img} for img in test_images[:]]
test_files[:5]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img"]),   
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"])
    ]
)
test_ds = monai.data.Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1,  collate_fn=list_data_collate)

In [ ]:
Pub_data = sorted(glob.glob(tempdir + "*.jpg"))
Pub_data[0].split("/")[-1].split(".")[0]

In [ ]:
model.eval()
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        test_images = test_data["img"].to(device)

        test_outputs, test_out_label = model(test_images) #forward
        test_outputs = Resize([-1, 1716, 942])(test_outputs)
        
        saverPD = SaveImage(output_dir=f"./Predict", output_ext=".png", output_postfix=f"{Pub_data[i].split('/')[-1].split('.')[0]}",scale=255,separate_folder=False)
        saverPD(test_outputs[0].cpu())

# Predict Private Dataset

In [ ]:
tempdir = "./Pravite_Image1/"
private_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))


print(f" {len(private_images)} private_images")

private_files = [{"img": img} for img in private_images[:]]
private_files[:5]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img"]),
        
#         AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),

        ScaleIntensityd(keys=["img"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"])
    ]
)
test_ds = monai.data.Dataset(data= private_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1,  collate_fn=list_data_collate)

In [ ]:
Pri_data = sorted(glob.glob(tempdir + "*.jpg"))
Pri_data[0].split("/")[-1].split(".")[0]

In [ ]:
model.eval()
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        test_images = test_data["img"].to(device)

        test_outputs, test_out_label = model(test_images) #forward
        test_outputs = Resize([-1, 1716, 942])(test_outputs)

        saverPD = SaveImage(output_dir=f"./Predict", output_ext=".png", output_postfix=f"{Pri_data[i].split('/')[-1].split('.')[0]}",scale=255,separate_folder=False)
        saverPD(test_outputs[0].cpu())

# 改檔名

> 改成官方所需格式`Public_00000000.jpg`、`Private_00000000.jpg` ...

In [ ]:
Predict = sorted(glob.glob(f"./Predict/*.png"))
print(len(Predict))

In [ ]:
for pred in Predict:
    os.rename(pred, os.path.join(*pred.split("/")[:-1], pred.split("/")[-1].split("_", 1)[-1]))